#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
from statsmodels.tsa.stattools import adfuller

import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from ta import add_all_ta_features




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [32]:
def load_and_setup_data(sybmol):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    #new_data = pd.DataFrame(input_data)
   # new_data.set_index("Date", inplace=True)
  #  concatenated_df = pd.concat([df,new_data])
   # print(concatenated_df)
    #print(concatenated_df)
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    #print(data)
    return data
    

In [33]:
#symbols = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]
symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","TITAN.NS","SUNPHARMA.BO","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]
#symbols = ["RELI","HDFCBANK.NS"]
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
#stock_data = fetch_stock_data(symbols)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    data = load_and_setup_data(symbol)
    # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    #data = DataProcessing(symbol,data)
    # Specify Target
    # data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    # data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    loaded_model_prophet = {}

    #feature_item = {}
    with open('../TrainedModel/xg/{}_model_phrophet.pkl'.format(symbol), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
   # with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
    #    feature_item = pickle.load(f)
    #final training
    loaded_models[symbol] = loaded_model
    data  = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    data = data.dropna()
    data["ds"] = data.index
   # data["Date"] =  pd.to_datetime(data['Date'])
    data["y"] = data["Close"]

    print(data)
    forecast = loaded_model_prophet.predict(data)
    data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
    
    #data = data.drop("index",axis=1)
    
    #get some variables
    prophet_variables = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","trend_upper","trend_lower"]]
    df_xgb = pd.concat([data, prophet_variables], axis = 1)
    non_stationaries = []
    for col in df_xgb.columns:
        if col != "multiplicative_terms" and col != "Date" and col != "ds" :
            #print(col)
            dftest = adfuller(df_xgb[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    #print(f"Non-Stationary Features Found: {len(non_stationaries)}")
    
    df_stationary = df_xgb.copy()

    df_stationary = df_stationary.drop(columns=["Date","ds"])
    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    #print(df_stationary)
    df_stationary = df_stationary[loaded_model.feature_names_in_]

   # score = loaded_model.predict(df_stationary)
    y_train_pred = loaded_model.predict_proba(df_stationary)[:,1]
    
    #print(y_train_pred)
        #df_xgb.head(1)
    #print(forecast)
   # final_target[symbol] = data.df["TARGET"]
   # df = data.df[feature_item]
   # finaldata[symbol]= df 
    stock_name_only = stock_name.replace(".NS","")
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
   # print(greater)
    #final_result[symbol] = [current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'xg',stock_name_only,data.iloc[-1].Close,round(greater,2)])

#print(result)
    

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


                   Open         High          Low        Close     Volume  \
Date                                                                        
2020-01-01   174.885252   175.834626   173.386236   173.835938    24780.0   
2020-01-02   177.833316   177.833316   172.386893   173.536133    62181.0   
2020-01-03   173.536134   175.884599   171.137713   171.537445    62012.0   
2020-01-06   172.386883   172.786614   165.391473   166.590683    56749.0   
2020-01-07   169.888525   178.882623   169.888525   175.384918   111270.0   
...                 ...          ...          ...          ...        ...   
2023-07-26  1075.000000  1190.099976  1053.099976  1190.099976  1967962.0   
2023-07-27  1225.550049  1260.000000  1190.099976  1213.599976  2267913.0   
2023-07-28  1219.949951  1224.400024  1162.500000  1177.250000   910161.0   
2023-07-31  1178.900024  1188.000000  1141.000000  1150.150024   618415.0   
2023-07-31  1164.599976  1208.800049  1155.000000  1158.449951  1025846.0   

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


                  Open        High         Low       Close     Volume  \
Date                                                                    
2020-01-01  544.715615  547.183863  542.057551  543.956177   615465.0   
2020-01-02  546.756694  546.756694  539.636848  543.006897   835523.0   
2020-01-03  544.952912  546.471847  537.928033  540.823425  1086282.0   
2020-01-06  537.358444  539.209615  530.665801  533.703613   944473.0   
2020-01-07  536.409117  538.924819  526.631208  528.150085   532397.0   
...                ...         ...         ...         ...        ...   
2023-07-26  680.000000  687.000000  677.099976  678.950012   504923.0   
2023-07-27  682.000000  683.500000  670.500000  675.049988   561343.0   
2023-07-28  675.000000  690.950012  671.450012  689.750000  1040732.0   
2023-07-31  689.000000  698.799988  684.549988  694.700012  1015347.0   
2023-07-31  698.750000  704.900024  690.650024  693.549988  1289359.0   

              volume_adi   volume_obv  volume_cmf 

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


...

                   Open         High          Low        Close      Volume  \
Date                                                                         
2020-01-01   652.431235   653.112615   647.618895   649.109436    746371.0   
2020-01-02   649.109501   655.114296   648.172611   652.473877   1133488.0   
2020-01-03   652.431213   664.270390   650.344445   660.182007   2121704.0   
2020-01-06   659.969147   663.333523   654.262545   656.178955   1734439.0   
2020-01-07   655.838180   663.077952   647.831823   661.885498   1937143.0   
...                 ...          ...          ...          ...         ...   
2023-07-26  1158.000000  1167.000000  1139.449951  1143.199951   3170213.0   
2023-07-27  1111.000000  1125.000000  1082.300049  1099.900024  17594818.0   
2023-07-28  1100.000000  1100.000000  1085.050049  1091.050049   4244321.0   
2023-07-31  1095.000000  1119.000000  1087.150024  1115.500000   4765771.0   
2023-07-31  1120.000000  1154.500000  1119.900024  1143.500000  

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


                   Open         High          Low        Close     Volume  \
Date                                                                        
2020-01-01  1743.620050  1755.648806  1736.802186  1746.541992   397285.0   
2020-01-02  1744.399097  1753.067547  1736.996869  1744.058228   411266.0   
2020-01-03  1733.490786  1733.490786  1701.592920  1705.829712  1318628.0   
2020-01-06  1692.729736  1692.778482  1649.923331  1662.731201  1871804.0   
2020-01-07  1666.480862  1695.213291  1666.480862  1679.532227  1033650.0   
...                 ...          ...          ...          ...        ...   
2023-07-26  3399.850098  3413.000000  3365.000000  3373.000000   959076.0   
2023-07-27  3390.000000  3414.000000  3351.000000  3369.399902   874078.0   
2023-07-28  3375.000000  3399.000000  3364.500000  3390.699951   553750.0   
2023-07-31  3323.199951  3388.000000  3323.149902  3377.350098   933820.0   
2023-07-31  3372.000000  3385.000000  3338.000000  3342.949951   697308.0   

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


                  Open        High         Low       Close     Volume  \
Date                                                                    
2020-01-01  311.205266  314.497925  308.590490  313.626343  1514293.0   
2020-01-02  313.384228  317.112693  310.963151  313.142120  3091351.0   
2020-01-03  313.577914  313.577914  306.992595  308.154724  1718884.0   
2020-01-06  305.297804  308.106248  300.600909  302.247253  1712158.0   
2020-01-07  304.087280  309.316801  302.150419  304.958862  2146089.0   
...                ...         ...         ...         ...        ...   
2023-07-26  873.950012  883.950012  869.549988  873.900024  1940116.0   
2023-07-27  875.950012  880.000000  847.700012  850.049988  3203072.0   
2023-07-28  854.250000  860.400024  842.000000  859.349976  2365708.0   
2023-07-31  859.349976  860.849976  852.500000  859.299988  1453376.0   
2023-07-31  861.900024  863.299988  853.099976  856.750000   987770.0   

              volume_adi   volume_obv  volume_cmf 

In [34]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '01-08-2023',
   'result': '0',
   'pre_close': '1158.449951171875',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'OLECTRA',
   'prob': '51.47'},
  {'cdate': '01-08-2023',
   'result': '0',
   'pre_close': '693.5499877929688',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'CONCOR',
   'prob': '54.16'},
  {'cdate': '01-08-2023',
   'result': '1',
   'pre_close': '539.7000122070312',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'ELGIEQUIP',
   'prob': '47.39'},
  {'cdate': '01-08-2023',
   'result': '0',
   'pre_close': '94.0',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'IOC',
   'prob': '78.38'},
  {'cdate': '01-08-2023',
   'result': '0',
   'pre_close': '130.4499969482422',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'BEL',
   'prob': '57.24'},
  {'cdate': '01-08-2023',
   'result': '0',
   'pre_close': '7185.0498046875',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'TATAELXSI',
   'prob': '50.12'}

In [35]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [36]:
final_target

{}

In [9]:
#for symbol in symbols:
   # retrained_model = loaded_models[symbol].fit(finaldata[symbol],final_target[symbol])
    #print(retrained_model)
    #with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
     #   pickle.dump(retrained_model, f)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=6, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eva